### Import lib

In [1]:
import os
os.chdir("/cerea_raid/users/dumontj/dev/coco2/dl")
import numpy as np
import sys
import pandas as pd
import xarray as xr
import tensorflow as tf
from tensorflow import keras
import keras.backend as K
import matplotlib.pyplot as plt
import seaborn as sns
from treeconfigparser import TreeConfigParser
import joblib

import models.reg as rg
import models.seg as sg
from data.Data import Data
from include.loss import pixel_weighted_cross_entropy

## Download func

In [2]:
path_res = "/cerea_raid/users/dumontj/dev/coco2/dl/res"

In [3]:
def get_data(ds: str, scaler):
    config = TreeConfigParser()
    config.readfiles(os.path.join(path_res, ds, "config.cfg"))
    shuffle_indices = np.fromfile(os.path.join(path_res, ds, "shuffle_indices.bin")).astype(np.int64)
    data = Data(config, shuffle_indices)
    data.prepare_input(scaler)
    data.prepare_output()
    print("xco2 valid mean", np.mean(data.y.valid[0]))
    return data

In [4]:
def get_model(path_w: str):
    model = keras.models.load_model(os.path.join(path_w, "weights_cp_best.h5"), compile=False)
    model.compile('adam', loss=pixel_weighted_cross_entropy)    
    return model

def get_scaler(path_w: str):
    scaler = joblib.load(os.path.join(path_w, "scaler.save"))
    return scaler

In [5]:
def eval_ds_with_model(data, model):
    x_valid = tf.convert_to_tensor(data.x.valid[0], np.float32)
    y_valid = tf.convert_to_tensor(data.y.valid, np.float32)
    loss_valid = model.evaluate(x=x_valid, y=y_valid, batch_size=64)
    return loss_valid

In [7]:
def eval_ds_with_baseline(data):
    x_valid = tf.convert_to_tensor(data.x.valid[0], np.float32)
    y_valid = tf.convert_to_tensor(data.y.valid, np.float32)
    rt = 0.9
    x_valid = x_valid.numpy()
    x_save = x_valid.copy()
    x_valid[x_valid<0] = 0    
    x_valid = x_valid.reshape(-1, x_valid.shape[1]*x_valid.shape[2])
    evo_diff_cummass_rt = np.sort(x_valid, axis=1).cumsum(axis=1) - rt * np.sum(
        x_valid, axis=1
    ).reshape(-1, 1)
    N_drop_val = (
        np.where(evo_diff_cummass_rt < 0, evo_diff_cummass_rt, -np.inf).argmin(
            axis=1
        )
    )
    x_valid[
        x_valid.argsort(axis=1).argsort(axis=1) < N_drop_val.reshape(-1, 1)
    ] = 0
    pred_baseline_2 = x_valid.reshape(-1, x_save.shape[1], x_save.shape[2], 1)
    pred_baseline_2[pred_baseline_2>0] = 1
    baseline_10p_valid = tf.convert_to_tensor(pred_baseline_2, np.float32)
    return model.compute_loss(y=y_valid, y_pred=baseline_10p_valid).numpy()

### Case choices

### SEG basic

In [8]:
names_ds = ["par", "ber", "pw"]
df_indices_ds = ["ds_" + case for case in names_ds]
keys_ds = ["seg_" + case for case in names_ds]

names_w = ["par", "ber", "pw", "ber_DA"]
df_indices_w = ["w_" + case for case in names_w]
keys_w = ["seg_" + case for case in names_w]

### SEG advanced

In [9]:
names_ds = ["ber", "PGIPW"]
df_indices_ds = ["ds_" + case for case in names_ds]
keys_ds = ["seg_" + case for case in names_ds]

names_w = ["ber", "ber_DA", "PGIPW", "PGIPW_DA", "PGIPW_DA2"]
df_indices_w = ["w_" + case for case in names_w]
keys_w = ["seg_" + case for case in names_w]

### Make df of results

In [10]:
df = pd.DataFrame(dict(zip(df_indices_w, [[-1.]*len(names_ds)]*len(names_w))))
df.index = df_indices_ds
for df_idx_w, key_w in zip(df_indices_w, keys_w):
    full_path_w = os.path.join("/cerea_raid/users/dumontj/dev/coco2/dl/res", key_w)
    model = get_model(full_path_w)
    scaler = get_scaler(full_path_w)
    for df_idx_ds, key_ds in zip(df_indices_ds, keys_ds):
        data = get_data(key_ds, scaler)
        loss_valid = eval_ds_with_model(data, model)
        df.loc[df_idx_ds, df_idx_w] = loss_valid  

2022-06-24 22:04:16.629760: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-24 22:04:17.103782: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14488 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:65:00.0, compute capability: 7.5


mean, var xco2 402.0255 12.423682
data.inp.train.shape[0] (6874, 160, 160, 1)
data.out.train.shape (6874, 160, 160, 1)
xco2 valid mean 0.12690587980800166


2022-06-24 22:04:34.100839: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8202
2022-06-24 22:04:34.537600: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


27/27 [==============================] - 4s 62ms/step - loss: 0.2094
mean, var xco2 402.8727 13.941546
data.inp.train.shape[0] (24194, 160, 160, 1)
data.out.train.shape (24194, 160, 160, 1)
xco2 valid mean 0.3894021287448883
95/95 [==============================] - 4s 42ms/step - loss: 0.5347
mean, var xco2 402.0255 12.423682
data.inp.train.shape[0] (6874, 160, 160, 1)
data.out.train.shape (6874, 160, 160, 1)
xco2 valid mean 0.12690587980800166
27/27 [==============================] - 1s 37ms/step - loss: 0.1715
mean, var xco2 402.8727 13.941546
data.inp.train.shape[0] (24194, 160, 160, 1)
data.out.train.shape (24194, 160, 160, 1)
xco2 valid mean 0.3894021287448883
95/95 [==============================] - 4s 37ms/step - loss: 0.7054
mean, var xco2 402.0255 12.423682
data.inp.train.shape[0] (6874, 160, 160, 1)
data.out.train.shape (6874, 160, 160, 1)
xco2 valid mean 0.12690587980800166
27/27 [==============================] - 1s 37ms/step - loss: 0.3749
mean, var xco2 402.8727 13.941546

In [11]:
df

,w_ber,w_ber_DA,w_PGIPW,w_PGIPW_DA,w_PGIPW_DA2
ds_ber,0.209408,0.171506,0.374899,0.402481,0.421684
ds_PGIPW,0.534741,0.705443,0.199857,0.198121,0.469269


#### Add baseline

In [12]:
df["10p_bl"] = [-1.]*len(df.index)
for df_idx_ds, key_ds in zip(df_indices_ds, keys_ds):
    config_file = os.path.join("/cerea_raid/users/dumontj/dev/coco2/dl/cfg", key_ds + ".cfg")
    config = TreeConfigParser()
    config.readfiles(config_file)
    data = Data(config)
    data.prepare_input()
    data.prepare_output()
    
    mean_bl = eval_ds_with_baseline(data)
    df.loc[df_idx_ds, "10p_bl"] = mean_bl

mean, var xco2 402.0255 12.423682
data.inp.train.shape[0] (6874, 160, 160, 1)
data.out.train.shape (6874, 160, 160, 1)
mean, var xco2 402.8727 13.941546
data.inp.train.shape[0] (24194, 160, 160, 1)
data.out.train.shape (24194, 160, 160, 1)


In [13]:
df

,w_ber,w_ber_DA,w_PGIPW,w_PGIPW_DA,w_PGIPW_DA2,10p_bl
ds_ber,0.209408,0.171506,0.374899,0.402481,0.421684,2.902987
ds_PGIPW,0.534741,0.705443,0.199857,0.198121,0.469269,3.628656
